# Amazon Bedrock AgentCore Gateway と Amazon Bedrock AgentCore Runtime の統合

## 概要
[Amazon Bedrock AgentCore Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html) は、既存の AWS Lambda 関数と API（OpenAPI および Smithy）をインフラやホスティングを管理することなく、フルマネージドな MCP サーバーに変換する方法を提供します。[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) は、AI エージェントやツールをデプロイして実行するための安全でサーバーレス、目的に特化したホスティング環境を提供します。このチュートリアルでは、Amazon Bedrock AgentCore Gateway を AgentCore Runtime と [Strands agents](https://strandsagents.com/latest/) に統合します。

### チュートリアル詳細


| 情報                 | 詳細                                                       |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ  | インタラクティブ                                           |
| AgentCore コンポーネント | AgentCore Gateway、AgentCore Identity、AgentCore Runtime |
| エージェントフレームワーク | Strands Agents                                        |
| Gateway ターゲットタイプ | AWS Lambda、OpenAPI ターゲット                           |
| インバウンド認証 IdP  | AWS IAM                                                   |
| アウトバウンド認証    | AWS IAM（AWS Lambda）、API キー（OpenAPI ターゲット）       |
| LLM モデル           | Anthropic Claude Haiku 4.5、Amazon Nova Pro               |
| チュートリアル構成    | AgentCore Gateway の作成と呼び出し                         |
| チュートリアル分野    | クロスバーティカル                                         |
| 例の複雑さ           | 中程度                                                     |
| 使用 SDK             | boto3                                                     |


### チュートリアルアーキテクチャ
このチュートリアルでは、AWS Lambda 関数と RESTful API で定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。AWS Sigv4 形式の AWS IAM 認証情報を使用したイングレス認証を示します。AgentCore Gateway ツールを使用する Strands Agent を AgentCore Runtime にデプロイします。

デモンストレーション目的で、[Amazon Bedrock](https://aws.amazon.com/bedrock/) モデルを使用した Strands Agent を使用します。

<center>

![runtime gateway](./images/runtime_gateway.png)

</center>

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
import os
import sys

if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()

# 共有 utils モジュールにアクセスするために2階層上に移動
# utils.py ファイルには複数のチュートリアルで使用されるヘルパー関数が含まれています
utils_dir = os.path.abspath(os.path.join(current_dir, "../.."))

# utils ディレクトリを Python のモジュール検索パスに追加
# これにより、親ディレクトリから utils をインポートできるようになります
sys.path.insert(0, utils_dir)

In [ ]:
# Gateway 管理用のヘルパー関数を含む utils モジュールをインポート
import utils

In [ ]:
# チュートリアルに必要なすべてのライブラリをインポート
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import (
    destroy_bedrock_agentcore,
)
from deploy_cloudformation import deploy_stack, delete_stack
from pathlib import Path
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from streamable_http_sigv4 import streamablehttp_client_with_sigv4
import boto3
import getpass
import json
import logging
import uuid

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
session = boto3.Session()

credentials = session.get_credentials()

region = session.region_name

cf_client = boto3.client("cloudformation", region_name=region)

agentcore_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=region,
)

identity_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=region,
)

s3_client = session.client("s3")

sts_client = session.client("sts")
account_id = sts_client.get_caller_identity()["Account"]

In [ ]:
# Define configuration variables for the tutorial resources

# CloudFormation stack configuration
stack_name = "customer-support-lambda-stack"  # Name of the stack that creates Lambda and DynamoDB
template_file = (
    "cloudformation/customer_support_lambda.yaml"  # Path to CloudFormation template
)

# Gateway and target names
gateway_name = "customer-support-gateway"  # Name for the AgentCore Gateway
open_api_target_name = (
    "DemoOpenAPITargetS3NasaMars"  # Name for the NASA API gateway target
)
lambda_target_name = "LambdaUsingSDK"  # Name for the Lambda function gateway target

# S3 bucket configuration for storing OpenAPI specifications
unique_s3_name = str(
    uuid.uuid4()
)  # Generate a globally unique identifier for the bucket
bucket_name = (
    f"agentcore-gateway-{unique_s3_name}"  # Prefix with 'agentcore-gateway' for clarity
)
file_path = (
    "openapi-specs/nasa_mars_insights_openapi.json"  # Local path to OpenAPI spec
)
object_key = "nasa_mars_insights_openapi.json"  # S3 object key (filename in the bucket)

# Credential provider name for NASA API authentication
api_key_credential_provider_name = (
    "NasaInsightAPIKey"  # Will store the NASA API key securely
)

# Agent configuration
agent_name = (
    "customer_support_gateway"  # Name for the deployed agent in AgentCore Runtime
)

# Store the unique S3 name for use in other notebooks if needed
%store unique_s3_name

## ステップ 1: AWS Lambda と Amazon DynamoDB のデプロイ

### AWS CloudFormation スタックリソース

[AWS CloudFormation](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/Welcome.html) テンプレートは以下の AWS リソースをデプロイします：

1. **AgentCoreRuntimeExecutionRole**：以下の権限を持つ AgentCore Runtime 実行ロール：
   - コンテナデプロイ用の ECR イメージアクセス
   - モニタリングとデバッグ用の CloudWatch Logs
   - 可観測性用の X-Ray トレーシング
   - AI 機能用の Bedrock モデル呼び出し
   - MCP ツール呼び出し用の Gateway 呼び出し
   - ワークロード ID トークン生成

2. **GatewayAgentCoreRole**：以下の権限を持つ AgentCore Gateway 実行ロール：
   - Lambda 関数呼び出し
   - OpenAPI 仕様用の S3 アクセス
   - 認証情報取得用の Secrets Manager アクセス
   - すべての AgentCore および Bedrock 操作
   - 混乱した代理人保護を含む強化された信頼ポリシー

3. **CustomerSupportLambda**：以下を提供するメイン Lambda 関数：
   - `get_customer_profile`：顧客 ID、メール、または電話番号で顧客情報を取得
   - `check_warranty_status`：シリアル番号を使用して製品保証を検証

4. **PopulateDataFunction**：DynamoDB テーブルに初期データを投入するカスタムリソース Lambda

5. **CustomerProfileTable**：以下の構成で顧客情報を格納：
   - プライマリキー：`customer_id`
   - 柔軟な検索用に `email` と `phone` のグローバルセカンダリインデックス
   - 5件のサンプル顧客プロファイルを含む

6. **WarrantyTable**：以下の構成で製品保証情報を格納：
   - プライマリキー：`serial_number`
   - 顧客ベースのクエリ用に `customer_id` のグローバルセカンダリインデックス
   - 8件のサンプル保証レコードを含む

スタックはチュートリアルで使用する3つの ARN をエクスポートします：
- `CustomerSupportLambdaArn`：Lambda 関数の ARN（Gateway ターゲット）
- `GatewayAgentCoreRoleArn`：Gateway 実行用 ARN
- `AgentCoreRuntimeExecutionRoleArn`：Runtime エージェントデプロイ用 ARN

すべてのリソースにはコスト追跡と管理のための適切なタグが含まれています。

In [ ]:
# CloudFormation スタックをデプロイ
lambda_arn, gateway_role_arn, runtime_execution_role_arn = deploy_stack(
    stack_name=stack_name,
    template_file=template_file,
    region=region,
    cf_client=cf_client,
)

## ステップ 2: インバウンドリクエストの認可に AWS IAM を使用した AgentCore Gateway の作成

AgentCore Gateway は、インバウンド認証機能において重要な進歩を遂げ、従来の OAuth のみのアプローチを超えて [AWS IAM](https://aws.amazon.com/iam/) サポートを導入しました。[MCP プロトコル](https://modelcontextprotocol.io/docs/getting-started/intro)仕様では伝統的に認証に OAuth トークンが必要とされていますが、この新機能により顧客は MCP サーバーへのインバウンドリクエストに AWS IAM 認証情報を使用できるようになり、重要なエンタープライズ要件に対応しています。

以前は、Gateway 開発者はインバウンドリクエストの認証に OAuth トークンベースの認証を必要とする CUSTOM_JWT のみを認可タイプとして設定できました。このリリースにより、開発者は認可タイプとして AWS_IAM を選択できるようになり、インバウンド MCP リクエストに AWS Signature Version 4（SigV4）認証を有効にできます。

この実装では、2つの重要な IAM コンポーネントが導入されています：

- Gateway リクエスト認可用の新しいアクション `bedrock-agentcore:InvokeGateway`
- 認可タイプに基づく Gateway 作成を制御するための新しい条件キー `bedrock-agentcore:GatewayAuthorizerType`

In [ ]:
# AWS IAM を認可タイプとして AgentCore Gateway を作成
# これがキーとなる機能 - 認証に CUSTOM_JWT ではなく AWS_IAM を使用
create_response = agentcore_client.create_gateway(
    name=gateway_name,
    roleArn=gateway_role_arn,
    protocolType="MCP",
    authorizerType="AWS_IAM",
    description="AgentCore Gateway with AWS Lambda target type using AWS IAM for ingress auth",
)
logger.info(f"Gateway を作成しました: {create_response}")

gateway_id = create_response["gatewayId"]
gateway_url = create_response["gatewayUrl"]
logger.info(f"Gateway ID: {gateway_id}")
logger.info(f"Gateway URL: {gateway_url}")

## ステップ 3: Bedrock AgentCore Gateway を使用してカスタマーサポート AWS Lambda を MCP ツールに変換

<center>

![prereq](./images/lambda_dynamodb.png)

</center>

In [ ]:
# Lambda 関数を Gateway ターゲットとして設定
# これにより Lambda 関数の操作が AI エージェントが使用できる MCP ツールに変換されます
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_arn,
            "toolSchema": {
                "inlinePayload": [
                    {
                        # 最初のツール: 顧客プロファイル情報を取得
                        "name": "get_customer_profile",
                        "description": "Retrieve customer profile using customer ID, email, or phone number",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                # 顧客は ID、メール、または電話番号で検索可能
                                "customer_id": {"type": "string"},
                                "email": {"type": "string"},
                                "phone": {"type": "string"},
                            },
                            # 最低限、customer_id が必須
                            "required": ["customer_id"],
                        },
                    },
                    {
                        # 2番目のツール: 製品の保証ステータスを確認
                        "name": "check_warranty_status",
                        "description": "Check the warranty status of a product using its serial number and optionally verify via email",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                # シリアル番号で製品を一意に識別
                                "serial_number": {"type": "string"},
                                # 顧客メールは確認に使用可能
                                "customer_email": {"type": "string"},
                            },
                            # シリアル番号は必須
                            "required": ["serial_number"],
                        },
                    },
                ]
            },
        }
    }
}

# Gateway が Lambda 関数に認証する方法を設定
# Gateway の IAM ロール（gateway_role_arn）を使用して Lambda を呼び出す
credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

# Gateway ターゲットを作成 - これにより Lambda 関数が MCP ツールとして利用可能になる
response = agentcore_client.create_gateway_target(
    gatewayIdentifier=gateway_id,
    name=lambda_target_name,
    description="Lambda Target using SDK",
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config,
)

## ステップ 4: Bedrock AgentCore Gateway を使用して NASA Open API を MCP ツールに変換


<center>

![prereq](./images/nasa.png)

</center>

NASA の Open API から気象データを取得する火星気象エージェントを作成します。Nasa Insight API への登録が必要です。[こちら](https://api.nasa.gov/)から登録してください。無料です！登録すると、メールで API キーが届きます。この API キーを使用して、OpenAPI ターゲット作成用の認証情報プロバイダーを設定します。



### ステップ 4.1 API 認証情報プロバイダーの作成 - Amazon Bedrock AgentCore Identity

In [ ]:
# Securely prompt the user for their NASA API Key
# getpass hides the input to prevent it from being displayed on screen
nasa_api_key = getpass.getpass(prompt="Enter your NASA API Key: ")

In [ ]:
if not nasa_api_key:
    logger.error(
        "NASA API キーが必要です。上のセルを実行して API キーを入力してください。"
    )
    raise ValueError("NASA API キーが必要です")

# AgentCore Identity に API キーを安全に保存するための認証情報プロバイダーを作成
# これにより Gateway はキーを公開せずに NASA API に認証できます
response = identity_client.create_api_key_credential_provider(
    name=api_key_credential_provider_name,
    apiKey=nasa_api_key,
)

logger.info(f"認証情報プロバイダーのレスポンス: {response}")

credential_provider_arn = response["credentialProviderArn"]
logger.info(f"アウトバウンド認証情報プロバイダー ARN: {credential_provider_arn}")

### ステップ 4.2: [NASA OpenAPI 仕様](./openapi-specs/nasa_mars_insights_openapi.json)をアップロードするための Amazon S3 バケットの作成

In [ ]:
try:
    # OpenAPI 仕様ファイルを保存するための S3 バケットを作成
    if region == "us-east-1":
        # us-east-1 では LocationConstraint は不要
        s3bucket = s3_client.create_bucket(Bucket=bucket_name)
    else:
        # 他のすべてのリージョンではリージョンを指定するために LocationConstraint が必要
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name, CreateBucketConfiguration={"LocationConstraint": region}
        )

    # OpenAPI 仕様 JSON ファイルを S3 にアップロード
    # Gateway はこのファイルを読み込んで NASA API の構造を理解します
    with open(file_path, "rb") as file_data:
        response = s3_client.put_object(
            Bucket=bucket_name, Key=object_key, Body=file_data
        )

    openapi_s3_uri = f"s3://{bucket_name}/{object_key}"
    print(f"アップロードされたオブジェクトの S3 URI: {openapi_s3_uri}")
except Exception as e:
    print(f"ファイルアップロードエラー: {e}")

### ステップ 4.3 アウトバウンド認証の設定と Gateway ターゲットの作成


In [ ]:
# NASA OpenAPI ターゲットを設定
# これにより NASA の Mars Insight API が MCP ツールに変換されます
nasa_openapi_s3_target_config = {
    "mcp": {"openApiSchema": {"s3": {"uri": openapi_s3_uri}}}
}

# NASA へのアウトバウンドリクエスト用の API キー認証を設定
# Gateway は NASA API へのすべてのリクエストに API キーを付与します
api_key_credential_config = [
    {
        "credentialProviderType": "API_KEY",
        "credentialProvider": {
            "apiKeyCredentialProvider": {
                # NASA は "api_key" という名前のクエリパラメータで API キーを期待
                "credentialParameterName": "api_key",
                # 先ほど作成した認証情報プロバイダーの ARN
                "providerArn": credential_provider_arn,
                # NASA API はクエリ文字列でキーを期待（ヘッダーではない）
                "credentialLocation": "QUERY_PARAMETER",  # オプション: "HEADER" または "QUERY_PARAMETER"
                # 注: credentialPrefix（"Basic" や "Bearer" など）はヘッダーベースの認証で使用
                # "credentialPrefix": " "  # ヘッダーベースのトークン認証を使用する場合はコメント解除
            }
        },
    }
]

# OpenAPI Gateway ターゲットを作成
# これにより NASA API のすべての操作が MCP ツールとして利用可能になります
response = agentcore_client.create_gateway_target(
    gatewayIdentifier=gateway_id,
    name=open_api_target_name,
    description="OpenAPI Target with S3Uri using SDK",
    targetConfiguration=nasa_openapi_s3_target_config,
    credentialProviderConfigurations=api_key_credential_config,
)

## ステップ 5: Strands Agent から MCP ツールを呼び出す

#### MCP クライアント SDK での AWS IAM 認証のサポート
MCP クライアント SDK での AWS IAM 認証のサポート

AgentCore Gateway へのインバウンドリクエストで AWS IAM 認証がサポートされるようになりましたが、現在のオープンソース MCP クライアント SDK は、特にストリーマブル HTTP 接続での SigV4 認証のサポートが限られていることに注意することが重要です。しかし、AWS は「Run Model Context Protocol (MCP) servers with AWS Lambda」プロジェクトを通じてソリューションを提供しています。このプロジェクトには、ストリーミング HTTP 接続での SigV4 認証のための重要な拡張機能が含まれています。

この実装は [AWS Labs GitHub リポジトリ](https://github.com/awslabs/run-model-context-protocol-servers-with-aws-lambda/tree/main)で利用可能であり、ストリーミング接続の認証ギャップを埋め、Strands や LangChain などの一般的なエージェントフレームワークとシームレスに統合できます。StreamableHTTPTransportWithSigV4 クラスは、ストリーミング機能を維持しながら AWS SigV4 署名を処理するように標準の MCP トランスポートレイヤーを拡張し、AgentCore Gateway の新しい IAM 認証機能と互換性があります。

In [ ]:
def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    AWS SigV4 認証付きのストリーマブル HTTP トランスポートを作成します。

    この関数は、AWS Signature Version 4 (SigV4) を使用してリクエストを認証する
    MCP クライアントトランスポートを作成します。標準の MCP クライアントは AWS IAM
    認証をネイティブにサポートしていないため、これはそのギャップを埋めるために必要です。

    Args:
        mcp_url (str): MCP Gateway エンドポイントの URL
        service_name (str): SigV4 署名用の AWS サービス名（通常 "bedrock-agentcore"）
        region (str): Gateway がデプロイされている AWS リージョン

    Returns:
        StreamableHTTPTransportWithSigV4: SigV4 認証が設定されたトランスポートインスタンス

    Example:
        >>> transport = create_streamable_http_transport_sigv4(
        ...     mcp_url="https://gateway-id.gateway.region.aws.dev/mcp",
        ...     service_name="bedrock-agentcore",
        ...     region="us-west-2"
        ... )
    """
    # 現在の boto3 セッションから AWS 認証情報を取得
    # これらの認証情報は SigV4 でリクエストに署名するために使用されます
    session = boto3.Session()
    credentials = session.get_credentials()

    # SigV4 署名機能を持つカスタムトランスポートを作成して返す
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    MCP クライアントからツールの完全なリストを取得し、ページネーションを処理します。

    MCP サーバーはページ分割されたレスポンスでツールを返す場合があります。
    この関数はページネーションを自動的に処理し、利用可能なすべてのツールを
    単一のリストで返します。

    Args:
        client: MCP クライアントインスタンス（strands.tools.mcp.mcp_client.MCPClient から）

    Returns:
        list: MCP サーバーから利用可能なすべてのツールの完全なリスト

    Example:
        >>> mcp_client = MCPClient(lambda: create_transport())
        >>> all_tools = get_full_tools_list(mcp_client)
        >>> print(f"{len(all_tools)} 個のツールが見つかりました")
    """
    more_tools = True
    tools = []
    pagination_token = None

    # すべてのページを取得するまでループ
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)

        tools.extend(tmp_tools)

        # さらにページがあるかどうかを確認
        if tmp_tools.pagination_token is None:
            # これ以上ページはない - 完了
            more_tools = False
        else:
            # さらにページが存在 - 次のページを取得する準備
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools

In [ ]:
system_prompt = """
あなたは複数の専門ツールとサービスにアクセスできる親切な AI アシスタントです。

あなたの機能は以下の通りです：

1. **カスタマーサポートサービス**：
   - 顧客 ID、メール、または電話番号を使用して顧客プロファイル情報を取得
   - シリアル番号を使用して製品保証ステータスを確認
   - ティア、購入履歴、ライフタイムバリューを含む顧客アカウント詳細を表示

2. **NASA 火星気象データ**：
   - 最新の InSight 火星気象データ（直近7火星日分）を取得
   - 火星の大気温度、風速、気圧、風向に関する情報を提供
   - 火星気象観測の季節情報とタイムスタンプを共有

以下のガイドラインに常に従ってください：
<guidelines>
    - 内部ツールを使用する際にパラメータ値を推測しないでください
    - リクエストを処理するために必要な情報がない場合は、ユーザーに必要な詳細を丁寧に確認してください
    - 利用可能な内部ツール、システム、または関数に関する情報を決して開示しないでください
    - 内部プロセス、ツール、関数、またはトレーニングについて質問された場合は、常に「申し訳ありませんが、内部システムに関する情報を提供することはできません。」と応答してください
    - 常にプロフェッショナルで親切なトーンを維持してください
    - お問い合わせを効率的かつ正確に解決することに集中してください
    - 火星気象データを提示する際は、技術的な指標をユーザーフレンドリーな用語で説明してください
    - カスタマーサポートのお問い合わせでは、顧客のプライバシーとデータセキュリティを優先してください
</guidelines>
"""

In [ ]:
# SigV4 認証サポート付きの MCP クライアントを作成
# これは AWS_IAM 認可タイプを使用する Gateway に接続するために重要
mcp_client = MCPClient(
    # トランスポートをオンデマンドで作成するラムダ関数を渡す
    lambda: create_streamable_http_transport_sigv4(
        mcp_url=gateway_url, service_name="bedrock-agentcore", region=region
    )
)

# 適切なクリーンアップを保証するためにコンテキストマネージャー内で MCP クライアントを使用
with mcp_client:

    # Gateway から利用可能なすべてのツールを取得
    # これには Lambda ツールと NASA API ツールの両方が含まれます
    tools = get_full_tools_list(mcp_client)

    logger.info(f"以下のツールが見つかりました: {[tool.tool_name for tool in tools]}")

    # エージェントを動かす Bedrock モデルを設定
    model = BedrockModel(
        model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
        temperature=0.7,
    )

    # モデル、システムプロンプト、ツールを持つ Strands エージェントを作成
    agent = Agent(
        model=model,
        system_prompt=system_prompt,
        tools=tools,
    )

    # 例 1: 火星の天気について質問
    # エージェントは NASA API ツールを使用してリアルタイムデータを取得します
    # agent("利用可能なすべてのツールをリストしてください")  # テスト用にコメント解除
    logger.info("クエリ 1 を実行中")
    agent("火星の北部の天気はどうですか？")

    # 例 2: 製品保証ステータスを確認
    # エージェントは Lambda 関数ツールを使用して DynamoDB にクエリを実行します
    logger.info("クエリ 2 を実行中")
    agent(
        "Gaming Console Pro デバイスを持っています。保証ステータスを確認したいです。保証シリアル番号は MNO33333333 です。"
    )

    # 例 3: 直接ツール呼び出し（エージェントの意思決定をバイパス）
    # これは LLM を経由せずに MCP ツールを直接呼び出す方法を示しています
    logger.info("直接ツール呼び出しを実行中")
    result = mcp_client.call_tool_sync(
        tool_use_id="get-customer-profile-1",  # このツール呼び出しの一意識別子
        # ツール名のフォーマット: <ターゲット名>___<操作名>
        # トリプルアンダースコアがターゲットと操作を区切ります
        name=lambda_target_name + "___get_customer_profile",
        arguments={"customer_id": "CUST005"},
    )

    print(json.dumps(result, indent=2))

## （オプション）ステップ 6: Amazon Bedrock AgentCore Runtime へのデプロイ

Amazon Bedrock モデルを使用した Strands Agent から始めましょう。他のすべても同じように動作します。

### ステップ 6.1: AgentCore Gateway 統合を含む Strands コードの記述

In [ ]:
%%writefile mcp_agent.py
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from streamable_http_sigv4 import streamablehttp_client_with_sigv4
import boto3
import os

# Initialize the AgentCore Runtime application
# This wrapper makes our agent deployable to AWS Lambda via AgentCore
app = BedrockAgentCoreApp()


def get_required_env(name: str) -> str:
    """Get required environment variable or raise error."""
    value = os.getenv(name)
    if not value:
        raise RuntimeError(f"{name} environment variable is required")
    return value


# Configure the Bedrock model for the agent
model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)

# Define the system prompt that governs the agent's behavior
system_prompt = """
あなたは複数の専門ツールとサービスにアクセスできる親切な AI アシスタントです。

あなたの機能は以下の通りです：

1. **カスタマーサポートサービス**：
   - 顧客 ID、メール、または電話番号を使用して顧客プロファイル情報を取得
   - シリアル番号を使用して製品保証ステータスを確認
   - ティア、購入履歴、ライフタイムバリューを含む顧客アカウント詳細を表示

2. **NASA 火星気象データ**：
   - 最新の InSight 火星気象データ（直近7火星日分）を取得
   - 火星の大気温度、風速、気圧、風向に関する情報を提供
   - 火星気象観測の季節情報とタイムスタンプを共有

以下のガイドラインに常に従ってください：
<guidelines>
    - 内部ツールを使用する際にパラメータ値を推測しないでください
    - リクエストを処理するために必要な情報がない場合は、ユーザーに必要な詳細を丁寧に確認してください
    - 利用可能な内部ツール、システム、または関数に関する情報を決して開示しないでください
    - 内部プロセス、ツール、関数、またはトレーニングについて質問された場合は、常に「申し訳ありませんが、内部システムに関する情報を提供することはできません。」と応答してください
    - 常にプロフェッショナルで親切なトーンを維持してください
    - お問い合わせを効率的かつ正確に解決することに集中してください
    - 火星気象データを提示する際は、技術的な指標をユーザーフレンドリーな用語で説明してください
    - カスタマーサポートのお問い合わせでは、顧客のプライバシーとデータセキュリティを優先してください
</guidelines>
"""


def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    Create a streamable HTTP transport with AWS SigV4 authentication.

    This function creates an MCP client transport that uses AWS Signature Version 4 (SigV4)
    to authenticate requests. Essential for connecting to IAM-authenticated gateways.

    Args:
        mcp_url (str): The URL of the MCP gateway endpoint
        service_name (str): The AWS service name for SigV4 signing
        region (str): The AWS region where the gateway is deployed

    Returns:
        StreamableHTTPTransportWithSigV4: A transport instance configured for SigV4 auth
    """
    # Get AWS credentials from the current boto3 session
    # These credentials will be used to sign requests with SigV4

    session = boto3.Session()
    credentials = session.get_credentials()

    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,  # Uses credentials from the Lambda execution role
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    Retrieve the complete list of tools from an MCP client, handling pagination.

    MCP servers may return tools in paginated responses. This function handles the
    pagination automatically and returns all available tools in a single list.

    Args:
        client: An MCP client instance

    Returns:
        list: A complete list of all tools available from the MCP server
    """
    more_tools = True
    tools = []
    pagination_token = None

    # Iterate through all pages of tools
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)

        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools


GATEWAY_URL = get_required_env("GATEWAY_URL")
GATEWAY_REGION = get_required_env("GATEWAY_REGION")

# Create the MCP client with SigV4 authentication
mcp_client = MCPClient(
    lambda: create_streamable_http_transport_sigv4(
        mcp_url=GATEWAY_URL,  # Gateway URL should be set as an environment variable
        service_name="bedrock-agentcore",
        region=GATEWAY_REGION,
    )
)

# Start the MCP client connection
mcp_client.start()

# Fetch all available tools from the gateway
tools = get_full_tools_list(mcp_client)

# Create the Strands agent with the model, system prompt, and tools
agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=tools,
)


@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Main entrypoint for the AgentCore Runtime deployed agent.

    This function is invoked when the agent receives a request through AgentCore Runtime.
    It extracts the user's prompt from the payload and returns the agent's response.

    Args:
        payload (dict): The incoming request payload containing the user's prompt
                       Expected format: {"prompt": "user's question"}

    Returns:
        str: The agent's text response after processing the prompt

    Example payload:
        {"prompt": "What is the weather on Mars?"}
    """
    # Extract the user's input from the payload
    user_input = payload.get("prompt")
    print("User input:", user_input)

    # Invoke the agent with the user's prompt
    # The agent will decide which tools to use (if any) to answer the question
    response = agent(user_input)

    # Extract and return the text content from the response
    return response.message["content"][0]["text"]


# Standard Python idiom: only run the app when this file is executed directly
if __name__ == "__main__":
    app.run()

### ステップ 6.2: Amazon Bedrock AgentCore Runtime の設定

まず、スターターツールキットを使用して、エントリーポイント、作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、スターターキットを起動時に Amazon ECR リポジトリを自動作成するように設定します。

設定ステップ中に、アプリケーションコードに基づいて Docker ファイルが生成されます。

<center>

![prereq](./images/configure.png)

</center>

In [ ]:
# AgentCore Runtime マネージャーを初期化
# このオブジェクトはエージェントの AWS Lambda へのデプロイを処理します
agentcore_runtime = Runtime()

# Runtime デプロイメント設定を構成
# これによりエージェントをデプロイするために必要なすべての AWS リソースが準備されます
response = agentcore_runtime.configure(
    entrypoint="mcp_agent.py",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    execution_role=runtime_execution_role_arn,
    non_interactive=True,
)

# 設定レスポンスを表示
response

### ステップ 6.3: Amazon Bedrock AgentCore Runtime の起動

Docker ファイルができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<center>

![prereq](./images/launch.png)

</center>

In [ ]:
# AgentCore Runtime を通じてエージェントを AWS Lambda に起動
# これによりコンテナイメージがビルドされ、ECR にプッシュされ、Lambda 関数が作成されます
# 注: このステップはコンテナのビルドとアップロードのため数分かかる場合があります
launch_result = agentcore_runtime.launch(
    env_vars={
        "GATEWAY_URL": gateway_url,
        "GATEWAY_REGION": region,
    }
)

### ステップ 6.4: AgentCore Runtime の呼び出し

最後に、ペイロードを使用して AgentCore Runtime を呼び出すことができます。

<center>

![prereq](./images/invoke.png)

</center>

In [ ]:
# サンプルプロンプトでデプロイされたエージェントをテスト
# これは AgentCore Runtime を通じて Lambda 関数を呼び出します
invoke_response = agentcore_runtime.invoke(
    {"prompt": "火星の北部の天気はどうですか？"}
)

# エージェントからのレスポンスを表示
print(invoke_response["response"][0])

In [ ]:
# デプロイされたエージェントの保証確認機能をテスト
# これにより Lambda ツール統合が正しく動作していることを確認します
invoke_response = agentcore_runtime.invoke(
    {
        "prompt": "Gaming Console Pro デバイスを持っています。保証ステータスを確認したいです。保証シリアル番号は MNO33333333 です。"
    }
)

# 保証確認のレスポンスを表示
print(invoke_response["response"][0])

## クリーンアップ

In [ ]:
# Gateway とすべての関連ターゲットを削除
# utils.delete_gateway 関数はターゲットの削除と Gateway の削除の両方を処理します
# 前のセルの実際の Gateway ID に置き換えてください

gateways = agentcore_client.list_gateways()

# Gateway のすべてのページを反復処理（ページネーションを処理）
while True:
    for gateway in gateways["items"]:
        if gateway["name"] == gateway_name:
            utils.delete_gateway(agentcore_client, gateway["gatewayId"])

    if "nextToken" not in gateways:
        break
    else:
        gateways = agentcore_client.list_gateways(nextToken=gateways["nextToken"])

In [ ]:
# ===================================================================
# 包括的なクリーンアップセクション
# このセルはチュートリアル中に作成されたすべての AWS リソースを削除します
# 不要な AWS 料金を避けるためにこれを実行してください
# ===================================================================

# クリーンアップ関数をインポート

# ステップ 1: API キー認証情報プロバイダーを削除
# これにより AWS Secrets Manager から NASA API キーが削除されます
try:
    logger.info(
        f"API キー認証情報プロバイダーを削除中: {api_key_credential_provider_name}"
    )
    identity_client.delete_api_key_credential_provider(
        name=api_key_credential_provider_name
    )
    logger.info("API キー認証情報プロバイダーを正常に削除しました")
except identity_client.exceptions.ResourceNotFoundException:
    logger.warning("API キー認証情報プロバイダーは存在しません")
except Exception as e:
    logger.error(f"API キー認証情報プロバイダーの削除エラー: {e}")

# ステップ 2: S3 バケットとそのすべてのコンテンツを削除
# バケット自体を削除する前にすべてのオブジェクトを削除する必要があります
try:
    logger.info(f"S3 バケットを削除中: {bucket_name}")

    objects = s3_client.list_objects_v2(Bucket=bucket_name)

    if "Contents" in objects:
        for obj in objects["Contents"]:
            s3_client.delete_object(Bucket=bucket_name, Key=obj["Key"])
            logger.info(f"オブジェクトを削除しました: {obj['Key']}")

    s3_client.delete_bucket(Bucket=bucket_name)
    logger.info(f"S3 バケットを正常に削除しました: {bucket_name}")
except s3_client.exceptions.NoSuchBucket:
    logger.warning(f"S3 バケット {bucket_name} は存在しません")
except Exception as e:
    logger.error(f"S3 バケットの削除エラー: {e}")

# ステップ 3: CloudFormation スタックを削除
# これにより Lambda 関数、DynamoDB テーブル、IAM ロールが削除されます
success = delete_stack(
    stack_name=stack_name, region=region, cf_client=cf_client, wait=True
)

if success:
    print("スタックを正常に削除しました")

In [ ]:
# ステップ 4: AgentCore Runtime とその他のリソースをクリーンアップ

destroy_bedrock_agentcore(
    config_path=Path(".bedrock_agentcore.yaml"),
    agent_name=agent_name,
    delete_ecr_repo=True,
)

In [ ]:
# Jupyter の変数ストアから保存された変数をクリーンアップ
# これらはノートブックの前半で %store を使用して保存されました
# ノートブックを再実行する際の競合を防ぐためにこれらを削除します
%store -d unique_s3_name

# おめでとうございます！